In [1]:
#!/usr/bin/env python

In [2]:
from subprocess import Popen, PIPE

from gtp import parse_vertex, gtp_move, gtp_color
from gtp import BLACK, WHITE, PASS

In [3]:
class GTPSubProcess(object):

    def __init__(self, label, args):
        self.label = label
        self.subprocess = Popen(args, stdin=PIPE, stdout=PIPE)
        print("{} subprocess created".format(label))

    def send(self, data):
        print("sending {}: {}".format(self.label, data))
        self.subprocess.stdin.write(data)
        result = ""
        while True:
            data = self.subprocess.stdout.readline()
            if not data.strip():
                break
            result += data
        print("got: {}".format(result))
        return result

    def close(self):
        print("quitting {} subprocess".format(self.label))
        self.subprocess.communicate("quit\n")

In [4]:
class GTPFacade(object):

    def __init__(self, label, args):
        self.label = label
        self.gtp_subprocess = GTPSubProcess(label, args)

    def name(self):
        self.gtp_subprocess.send("name\n")

    def version(self):
        self.gtp_subprocess.send("version\n")

    def boardsize(self, boardsize):
        self.gtp_subprocess.send("boardsize {}\n".format(boardsize))

    def komi(self, komi):
        self.gtp_subprocess.send("komi {}\n".format(komi))

    def clear_board(self):
        self.gtp_subprocess.send("clear_board\n")

    def genmove(self, color):
        message = self.gtp_subprocess.send(
            "genmove {}\n".format(gtp_color(color)))
        assert message[0] == "="
        return parse_vertex(message[1:].strip())

    def showboard(self):
        self.gtp_subprocess.send("showboard\n")

    def play(self, color, vertex):
        self.gtp_subprocess.send("play {}\n".format(gtp_move(color, vertex)))

    def final_score(self):
        self.gtp_subprocess.send("final_score\n")

    def close(self):
        self.gtp_subprocess.close()

In [5]:
GNUGO = ["gnugo", "--mode", "gtp"]
GNUGO_LEVEL_ONE = ["gnugo", "--mode", "gtp", "--level", "1"]
GNUGO_MONTE_CARLO = ["gnugo", "--mode", "gtp", "--monte-carlo"]

In [6]:
black = GTPFacade("black", GNUGO)
white = GTPFacade("white", GNUGO_LEVEL_ONE)

black subprocess created
white subprocess created


In [7]:
black.name()
black.version()

sending black: name



TypeError: a bytes-like object is required, not 'str'

In [8]:
white.name()
white.version()

sending white: name



TypeError: a bytes-like object is required, not 'str'

In [9]:
black.boardsize(9)
white.boardsize(9)

sending black: boardsize 9



TypeError: a bytes-like object is required, not 'str'

In [10]:
black.komi(5.5)
white.komi(5.5)

sending black: komi 5.5



TypeError: a bytes-like object is required, not 'str'

In [11]:
black.clear_board()
white.clear_board()

sending black: clear_board



TypeError: a bytes-like object is required, not 'str'

In [12]:
first_pass = False

In [13]:
while True:
    vertex = black.genmove(BLACK)
    if vertex == PASS:
        if first_pass:
            break
        else:
            first_pass = True
    else:
        first_pass = False

    black.showboard()

    white.play(BLACK, vertex)
    white.showboard()

    vertex = white.genmove(WHITE)
    if vertex == PASS:
        if first_pass:
            break
        else:
            first_pass = True
    else:
        first_pass = False

    white.showboard()

    black.play(WHITE, vertex)
    black.showboard()

sending black: genmove B



TypeError: a bytes-like object is required, not 'str'

In [14]:
black.final_score()
white.final_score()

sending black: final_score



TypeError: a bytes-like object is required, not 'str'

In [15]:
black.close()
white.close()

quitting black subprocess


TypeError: memoryview: a bytes-like object is required, not 'str'